# 2. Pipeline de Clasificación Híbrida
Este notebook implementa el pipeline completo de clasificación, incluyendo:
1. Preprocesamiento de texto biomédico
2. Generación de embeddings
3. Clasificación y evaluación

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
sys.path.append('..')

# Importar componentes del proyecto
from src.preprocessing.cleaner import TextCleaner
from src.preprocessing.normalizer import SemanticNormalizer
from src.features.embeddings import BiomedicalEmbeddings
from src.models.classifier import BiomedicalClassifier
from src.evaluation.metrics import ModelEvaluator</VSCode.Cell>

<VSCode.Cell language="markdown">
## 1. Cargar y Preparar Datos

In [ ]:
# Cargar datos
df = pd.read_csv('../data/raw/challenge_data-18-ago.csv')

# Split en train/test
X_train, X_test, y_train, y_test = train_test_split(
    df[['title', 'abstract']],
    df['group'],
    test_size=0.2,
    random_state=42,
    stratify=df['group']
)</VSCode.Cell>

<VSCode.Cell language="markdown">
## 2. Preprocesamiento de Texto

In [ ]:
# Inicializar componentes
classifier = BiomedicalClassifier()

# Preprocesar textos de entrenamiento
print("Preprocesando textos de entrenamiento...")
X_train_processed = classifier.preprocess_text(
    X_train['title'].tolist() + X_train['abstract'].tolist()
)

# Mostrar ejemplo de preprocesamiento
print("\nEjemplo de preprocesamiento:")
idx = 0
print("Original:")
print(f"Título: {X_train['title'].iloc[idx]}")
print(f"Abstract: {X_train['abstract'].iloc[idx]}")
print("\nProcesado:")
print(f"Texto: {X_train_processed[idx]}")</VSCode.Cell>

<VSCode.Cell language="markdown">
## 3. Generación de Embeddings

In [ ]:
# Generar embeddings
print("Generando embeddings...")
X_train_features = classifier.get_features(
    X_train['title'].tolist(),
    X_train['abstract'].tolist()
)

print(f"\nDimensiones de los features: {X_train_features.shape}")</VSCode.Cell>

<VSCode.Cell language="markdown">
## 4. Entrenamiento del Modelo

In [ ]:
# Entrenar modelo
print("Entrenando modelo...")
classifier.fit(
    X_train['title'].tolist(),
    X_train['abstract'].tolist(),
    y_train.tolist()
)</VSCode.Cell>

<VSCode.Cell language="markdown">
## 5. Evaluación

In [ ]:
# Inicializar evaluador
evaluator = ModelEvaluator(classifier)

# Realizar predicciones en test
predictions, probas = classifier.predict(
    X_test['title'].tolist(),
    X_test['abstract'].tolist()
)

# Calcular métricas
metrics = evaluator.calculate_metrics(y_test.tolist(), predictions, probas)
print("\nMétricas de evaluación:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Visualizar matriz de confusión
evaluator.plot_confusion_matrix(y_test.tolist(), predictions)

# Analizar errores
errors = evaluator.analyze_errors(
    y_test.tolist(),
    predictions,
    X_test['title'].tolist()
)

print("\nEjemplos de errores:")
for i, error in enumerate(errors[:5]):
    print(f"\nError {i+1}:")
    print(f"Texto: {error['text']}")
    print(f"Etiquetas reales: {error['true_labels']}")
    print(f"Predicciones: {error['predicted_labels']}")</VSCode.Cell>

<VSCode.Cell language="markdown">
## 6. Explicabilidad